# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is a classification problem. The key reasoning is since we want to predict which student would have the most likely chances of failing to graduate, the output here is a discrete output; either the student will fail or not fail. If an output is instead continuous, for example, predicting a student's test score of a particular subject (X over 100%), then it is considered as a regression problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] == "yes"])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == "no"])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/float(n_students)*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size = num_train, test_size = num_test, random_state = 42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
1) Support Vector Machines
Real-world application: Cancer classification based on feature selection of the genes which contribute most to cancer. In this reference paper, the author used SVM to classify Leukemia and Colon cancer.
Reference: Gene Selection for Cancer Classification using Support Vector Machines, 2002, GUYON et al http://ace.cs.ohiou.edu/~razvan/courses/ml6830/papers/guyon-ml02.pdf

Strengths:
SVM is good for modelling complex and non-linear domains. For example, the student's graduation is modelled on internal factors (student's grades, age, gender, study time, etc) and external factors (who is the guardian, guardian's job, type of school, etc). Where, the underlying interrelationships between these factors are often complex (not a simple "If X increases, Y decreases proportionally" model). Also, SVM performs well on data which has a clear margin of separation, where the classes do not overlap. 

Weaknesses:
SVM becomes computationally expensive if the data set is large (> 100k samples). Hence, training and predicting speeds will be slow on large datasets. SVM does not work well on skewed classes, where one class is many times greater in porportion to the other class.

Reasoning for pick:
Given that the data is a binary classification problem, SVM would work well. Also, SVM will perform well on the complex nature of modelling student's graduation (interrelationship among internal and external factors). The dataset is also not too heavily skewed, where about 67% of the students passed. 

2) k-Nearest Neighbors
Real-world application: Financial modelling of financial indicators using k-NN
Reference: Optimal Pruned K-Nearest Neighbors: OP-KNN –Application to Financial Modeling, 2008, Yu et al http://research.ics.aalto.fi/eiml/Publications/Publication110.pdf

Strengths:
k-NN is fast to train, as it is instance-based. It only consults the data closests to the query point, considered as a 'lazy' learner. As a result, we do not need to train the model on the full dataset at a time and new data points can easily be added to the dataset without any need to retrain the model. k-NN is simple and straightforward to implement. 

Weaknesses:
The output of k-NN on it's prediction will vary widely on the parameters specified, especially k, the number of nearest neighbors for the model to consider. Thus, the selection of the parameters will need to depend on the particular dataset to determine the appropriate domain knowledge to be used. k-NN can also be slow on large datasets.

Reasoning for pick:
Although the predicting speed of k-NN might be slow, in the case of predicting the student failures, it is not imperative to be able to produce the predictions quickly (say, in milliseconds), as the speed is not a critical consideration in this application. Also, it is a simple model to implement and tune.

3) Random Forest
Real-world application: Classifying vehicle type on a traffic measurement system. Application of random forest algorithm to classify vehicles detected by a multiple inductive loop system, 2012, Sheik Mohammed Ali S et al
http://ieeexplore.ieee.org/document/6338719/

Strengths: 
Random Forest performs well on modelling datasets for classification and generally has higher accuracy than other models since it is an ensemble method of algorithm. It can handle noise data fairly well as well as noise variables as along as the noise-to-signal ratio is not too high. Random Forest can aid in feature selection, outputting which features contribute most to the prediction results.

Weaknesses:
Random Forest may not be visually easy to interpret, as multiple trees are grown with random feature selection. For best performance, it also requires some tuning of its parameters.

Reasoning for pick:
Random Forest is an ensemble method and has good performance in comparison to other algorithms. Given the multiple features of the dataset, it would be useful to know which features are most relevant in predicting the student pass or fail outcome as well.  

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# TODO: Initialize the three models
clf_A = SVC(random_state = 42)
clf_B = KNeighborsClassifier()
clf_C = RandomForestClassifier(random_state = 42)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a SVC using a training set size of 100. . .
Trained model in 0.0140 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8777.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7746.
Training a SVC using a training set size of 200. . .
Trained model in 0.0040 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 0.8679.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7815.
Training a SVC using a training set size of 300. . .
Trained model in 0.0110 seconds
Made predictions in 0.0170 seconds.
F1 score for training set: 0.8761.
Made predictions in 0.0060 seconds.
F1 score for test set: 0.7838.
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 0.8060.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7246.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.00

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Support Vector Machines**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0140                  |  0.0010                | 0.8777           | 0.7746          |
| 200               | 0.0040                  |  0.0010                | 0.8679           | 0.7815          |
| 300               | 0.0110                  |  0.0060                | 0.8761           | 0.7838          |

** Classifer 2 - K-Nearest Neighbors**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0010                  |  0.0020                | 0.8060           | 0.7246          |
| 200               | 0.0010                  |  0.0020                | 0.8800           | 0.7692          |
| 300               | 0.0010                  |  0.0030                | 0.8809           | 0.7801          |

** Classifer 3 - Random Forest**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0350                  |  0.0020                | 0.9844           |  0.7153         |
| 200               | 0.1730                  |  0.0020                | 0.9964           |  0.7857         |
| 300               | 0.0500                  |  0.0030                | 0.9951           |  0.8092         |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** 
Choosing an appropriate model as the best model is often a tradeoff between performance and other factors. Performance-wise, Random Forest is the best, once we get to utilize 300 training examples. However, based on the amount of available data, if we are only expecting very small amounts of data to work with, Random Forest may not be a good candidate, as it performs poorly in these conditions. In smaller datasets, SVM may be a better choice. If we are expecting a large dataset to work with (> 100k samples), care must be taken that models such as the SVM will generally perform slower as it gets computationally costly to train and test the model. In terms of training and predicting times, both Random Forest and SVM are generally slower, compared to k-NN, but for this particular problem, we are more interested in getting more correct predictions than faster models. In summary, once we consider all factors, the model with the highest prediction power (Random Forest) outweighs the cons of slower testing and training times. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **
First, we feed Random Forest a dataset of students of the various features (age, gender, school type, etc) alongside the labels 'passed' and 'failed' for each student. The model works in principle, to build many smaller models called Trees which is then combined into one big model of a 'Forest' of Trees. Each Tree asks questions pertaining on the dataset, such as "Is student age > 15?" and "Does student use the Internet?". The goal for each Tree is to separate the students based on asking such questions. Each small Tree model may not be very powerful in predicting the student's outcomes, but the prediction power increases greatly with the resulting Random Forest model. In other words, Random Forest leverages on the power of many. The model first takes a random sample from the training data and then takes a random set of features to be trained on, and grows a tree based on that selection. It repeats by selecting another random training data sample and feature sample and grows more trees, until the model sufficiently fits the overall dataset. Then the results of all trees is averaged or voted to make the final result. When it comes to predicting the graduation of new students, the Tree questions of the Random Forest model that was previously created and trained is now used to categorize the new data. 

================================================================================================================

When an SVM model is used to approach a problem, below are the steps that happen:
Let's say that the students who passed are represented by blue dots, and the students who fail by red dots. Each dot represents a student, and we gather all of these dots onto a flat soccer field. Now, we want to separate them as best as possible, using a straight line of a ruler. From our top-down view, we could see that we can visually separate these students fairly easily with a line, but there is one problem; there are muliple ways to align the ruler to sepaprate them. 

http://houxianxu.github.io/images/SVM/1.png
(Lines 1, 2 and 3. Many lines to separate the dots)

How do we know if the line we choose is the best line? We start by reasoning "Well, the best line can be thought of as the line that clearly separates between blue and red dots, and in such a way that the dots are as far as possible from the line we choose. This gives us most confidence that the line is indeed able to distinguish the two types of dots. We the place two more rulers on the line, and we push each ruler away from the middle line, up until one ruler hits one of the red dots and the other ruler hits one of the blue dots, forming a 'street' along these groups of dots. We adjusted the 'street' until we get the widest possible straight 'street' we can. Those few dots which touches the street boundaries are called 'supports' to our model, as they are key in determining the street margins.

http://research.microsoft.com/en-us/um/people/manik/projects/trade-off/figs/svm2.PNG
(Street margins and boundaries)

Now all is good, but it turns out there is a late comer to the field, so one student came running in and stood at a spot. Unfortunately, where the student stood caused a problem; our current 'street' is not separating all reds from blues anymore. No matter how much we try adjusting the straight line, there always seems to have a few blue dots in the red territory and red dots in the blue territory. Now we cant bend the ruler in this world, as it will break into pieces, but what we can do is bring out our secret gadget that turns our soccer field into a 3D cube-like field, where the students are suspended in various positions in mid-air. We can't use our rulers here, as it is meant for the 2D world, but we have a sheet of metal, which we use to slice through the 3D space so that all red and blue dots are now again separated. We are satisfied with what we've managed to do, and turned off the gadget so that the field returns to a flat piece again. Now, when we look at the outcome, we see that the sheet of metal has transformed into a curvy line that snaked through the groups of dots, and so we've managed to separate blues and reds again! Our 'street' is now curved!

http://i.stack.imgur.com/1gvce.png
(Separating dots in higher dimensions)

We leave our separating lines on the field and dispersed the students. Come next day, a batch of new students arrived. This time, we dont know if they are blue or red, because their teacher refused to tell us. We decided to employ our decision lines we drew previously and use that to group the students. We asked the students to scatter on the field and then we asked each one a few questions. We want to find the few students who would most likely exhibit similar characteristics of our 'support' students. We found a few matches, and marked them. Using them as bearings, we take our earlier 'street' model and place it where the new 'support' dots matched up as closely as our old ones previously. We decide that dots who are at one side of the street are red and the other side, blue, predicting the results based on what we've learnt previously.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [27]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'max_features': [1, 2, 3, 4, 5, 'auto']}

# TODO: Initialize the classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label = 'yes', greater_is_better = False)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator = clf, param_grid = parameters, scoring = f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0020 seconds.
Tuned model has a training F1 score of 0.9831.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.7704.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
The final models F1 Score for Training and Testing are 0.9831 and 0.7704 respectively. As noticed, these scores are lower than the untuned model. There could be a few reasons due to this. One of the reasons is by using GridSearchCV implementation, we are using k-fold cross validation of k=3 by default. Whereas in the untuned model, we utilised the train_test_split defined by the size of training and testing splits. This means, a cross validated k-fold will typically produce lower F1 scores, as it is validated over multiple folds vs a single fold in the untuned model. In real-world applications, k-fold might reflect a better performance score compared to non k-fold.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.